<a href="https://colab.research.google.com/github/lakshitgosain/Tensorflow-ZTM/blob/main/TF_ZTM_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundamentals in TF

Sequence-to-sequence
* One to one- Image Captioning
* Many to one- Sentiment analysis
* Many to one- time series forecasting
* many- to-many- Machine translation


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ac9eb8d1-7ffa-373f-0123-bc43270c9b3a)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-15 07:54:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-15 07:54:54 (87.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-06-15 07:54:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.253.114.128, 172.253.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2023-06-15 07:54:58 (84.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualize the dataset

To visualize the sample data, we need to read the in. On way is to use python.

Another way to do this is use pandas



In [6]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [7]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_data_shuffled=train_df.sample(frac=1, random_state=42)

In [9]:
train_data_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [10]:
#Test dataframe looks like:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
#How many total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [13]:
#Let's visualize some random training samples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_data_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text, target=row
  print(f"Target: {target}", "(real disaster)" if target >0 else "(not real disaster)")
  print(f"text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
text:
@annajhm @JCOMANSE @paul_staubs @rslm72254 @blanktgt You must've got in trouble ??that's why you have #Fartanxiety now????

---

Target: 1 (real disaster)
text:
@115Film Doctor we must leave immediately the Core is unstable...The whole building is told to be evacuated. Take the research. We need...

---

Target: 1 (real disaster)
text:
@irishspy What you don't think the Allies should have just sucked up 1 million casualties?

---

Target: 1 (real disaster)
text:
People are more worried about the burning of buildings than black people losing their lives disgusting.

---

Target: 1 (real disaster)
text:
Only one man Tsutomi Yamaguchi is said to have survived both atomic bomb blasts at #Hiroshima and #Nagasaki. #OTD http://t.co/DaalPeNZp0

---



### Split data into training and validation sets



In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences, val_sentences, train_labels, val_labels=train_test_split(train_data_shuffled['text'].to_numpy(),
                                                                          train_data_shuffled['target'].to_numpy(),
                                                                          test_size=0.1,
                                                                          random_state=42)

In [16]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Converting words into numbers

First thing while building a model is to convert your text into numbers


Tokenization VS Embedding
Tkenization - word level and character level. Direct apping of a token.

Embedding - every word gets turned into a vector and we can define size of the vector. Embeddings can learn as our model trains



Text vectorization(tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
#Use the default text vectorization parameters
text_vectoizer= TextVectorization(max_tokens=None, #Defines how many words in our vocab
                               standardize= "lower_and_strip_punctuation",
                               split="whitespace",
                               ngrams=None,
                               output_mode='int',
                               output_sequence_length=None) #how long do you want your sequences to be
                               #pad_to_max_tokens=True)

In [19]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [20]:
#Find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
max_vocab_length= 10000 #max no of words to have in our vocab
max_length=15 #Max length our sequences will be (e.g. how many words from a Tweet does a model see)

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                     output_mode="int",
                                     output_sequence_length=max_length)

In [22]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [23]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [24]:
#Create a sample sentence and tokenize it
sample_sentence="There is a floow in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3,   1,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
random_sentence=random.choice(train_sentences)
print(random_sentence)
vectorized_random_sentence=text_vectorizer([random_sentence])
print(f"Random Sentence: {random_sentence}\nVectoized Random Sentence: {vectorized_random_sentence}")

#Afghanistan: sharp rise in women and children casualties in first half of #2015 http://t.co/LdyWd4ydT9
Random Sentence: #Afghanistan: sharp rise in women and children casualties in first half of #2015 http://t.co/LdyWd4ydT9
Vectoized Random Sentence: [[1439 2300  727    4  427    7  656  518    4   97  575    6  208    1
     0]]


In [26]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
botton_5_words=words_in_vocab[-5:]

In [27]:
words_in_vocab,top_5_words,botton_5_words

(['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with',
  'it',
  'that',
  'at',
  'by',
  'this',
  'from',
  'be',
  'are',
  'was',
  'have',
  'like',
  'as',
  'up',
  'so',
  'just',
  'but',
  'me',
  'im',
  'your',
  'not',
  'amp',
  'out',
  'its',
  'will',
  'an',
  'no',
  'has',
  'fire',
  'after',
  'all',
  'when',
  'we',
  'if',
  'now',
  'via',
  'new',
  'more',
  'get',
  'or',
  'about',
  'what',
  'he',
  'people',
  'news',
  'been',
  'over',
  'one',
  'how',
  'dont',
  'they',
  'who',
  'into',
  'were',
  'do',
  'us',
  '2',
  'can',
  'video',
  'emergency',
  'there',
  'disaster',
  'than',
  'police',
  'would',
  'his',
  'still',
  'her',
  'some',
  'body',
  'storm',
  'crash',
  'burning',
  'suicide',
  'back',
  'man',
  'california',
  'why',
  'time',
  'them',
  'had',
  'buildings',
  'rt',
  'first',
  'cant',
  'see',
  'got',
  'day',
  'off',
  'our',
  'going',
 

### Creating an embedding using an embedding layer

To make our embedding, we're going to use the TF's embedding layer.

The parameters we care the most about our embedding layers are:
* input_dim= sie of our vocab
* output_dim= size of the output embedding vector. A value of a 100 will mean that each token will be represented as a vector of 100 long length
* input_length= length of sequences being passed to the embedding layer
*

In [28]:
from tensorflow.keras import layers

In [30]:
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           input_length=max_length)

In [31]:
embedding

In [32]:
#Get a random sentence from the training set
random_sentence=random.choice(train_sentences)
print(f"Original Sentence:{random_sentence}\
      \nEmbedded Sentence")

sample_embedding=embedding(text_vectorizer([random_sentence]))
sample_embedding

Original Sentence:MaFireEMS: RT WMUR9: Two buildings involved in fire on 2nd Street in #Manchester. WMUR9  http://t.co/QUFwXRJIql via KCarosaWMUR      
Embedded Sentence


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01783091,  0.02592481,  0.0186563 , ...,  0.02584991,
         -0.02341678,  0.02544465],
        [ 0.02098122, -0.03860571,  0.03564078, ...,  0.02438277,
          0.00278072, -0.02660488],
        [-0.01250132,  0.04893612,  0.01884257, ..., -0.03141469,
          0.03925247, -0.04230182],
        ...,
        [-0.03282436,  0.03768099, -0.02799851, ...,  0.00510786,
         -0.0426378 ,  0.01679177],
        [-0.01250132,  0.04893612,  0.01884257, ..., -0.03141469,
          0.03925247, -0.04230182],
        [ 0.01783091,  0.02592481,  0.0186563 , ...,  0.02584991,
         -0.02341678,  0.02544465]]], dtype=float32)>

In [34]:
#Check out a single token's embedding
sample_embedding[0][0], sample_embedding[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01783091,  0.02592481,  0.0186563 , -0.01002058, -0.03591248,
        -0.04636083,  0.00280815, -0.02864938, -0.01817731, -0.03952376,
        -0.00392552,  0.01689071,  0.02733884, -0.03937083,  0.0292857 ,
         0.03110827,  0.03497079,  0.04608897,  0.00704961, -0.02222641,
        -0.01527553,  0.03682159,  0.00765996, -0.01493281,  0.00495691,
         0.02261562, -0.03590884, -0.04934757,  0.04630036,  0.02650198,
        -0.01972774,  0.01520215,  0.03853578,  0.00094329,  0.04524038,
         0.00766432,  0.04982356, -0.00425726,  0.03033056,  0.01132816,
        -0.02919952, -0.02731879,  0.02447276,  0.00015778,  0.006719  ,
         0.00910298, -0.00546825,  0.02265335, -0.02276393, -0.01733585,
         0.03793552, -0.01228113,  0.01318524,  0.01964075,  0.03335306,
        -0.03348223,  0.03906039, -0.03872584,  0.01063757,  0.00022243,
        -0.04300516,  0.00541227,  0.04874512,  0.03780986, -0.00805902,
  

##Modelling a text dataset(running a series of experiments)

* Model 0: Naive Bayes with Tokenization
* Model 1: Feed-forward Neural Network(Dense Model)
* Model 2: LSTM Model(RNN)
* Model 3: GRU Model
* Moel 4: Bidirectional LSTM
* Model 5 : 1D Convolutional Layer
* model 6: Tensorflow hub pretrained feature extraction (using transfer learning)
* Model 7 : Same as model 6 with 10% of data

How are we going to approach all of these:
Use the standard steps in modelling with tensorflow

* Create->build->fit->Evaluate

